In [1]:
import cx_Oracle
import pandas as pd

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select * from PC.L_VM_COMPLETA WHERE PERIODO <= 201910" 
df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_full = pd.concat(df, ignore_index=True)
conn.close()
print(df_full.shape)
# Imprimir las primeras filas del DataFrame completo
print(df_full.head())
# Imprimir los tipos de datos de las columnas del DataFrame completo
print(df_full.dtypes)
# Imprimir el número de filas y columnas del DataFrame completo
print(f"Número de filas: {df_full.shape[0]}, Número de columnas: {df_full.shape[1]}")

C:\Users\pcablinski.RWD\AppData\Local\Temp\ipykernel_30064\144710793.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


(10950473, 30)
   PERIODO  ANIO  MES  TRIMESTRE  ID_CAT1  ID_CAT2  ID_CAT3  ID_BRAND  \
0   201801  2018    1          1        2       11       57         1   
1   201802  2018    2          1        2       11       57         1   
2   201803  2018    3          1        2       11       57         1   
3   201804  2018    4          2        2       11       57         1   
4   201805  2018    5          2        2       11       57         1   

   SKU_SIZE  CUSTOMER_ID  ...  TN_LAG_03  TN_LAG_06  TN_LAG_12      CLASE  \
0      3000        10001  ...  222.11389  101.20711   99.43861  214.72336   
1      3000        10001  ...  111.54944   43.33930  198.84365  132.83419   
2      3000        10001  ...  131.27150  289.35024   92.46537  165.73507   
3      3000        10001  ...   49.61857  222.11389   13.29728   92.29410   
4      3000        10001  ...   88.44065  111.54944  101.00563  256.05760   

   CLASE_DELTA  TN_DELTA_01  TN_DELTA_02  TN_DELTA_03  TN_DELTA_06  \
0    165.1047

In [2]:
# Variables categóricas
categorical_features = ['CUSTOMER_ID','PRODUCT_ID','ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','PLAN_PRECIOS_CUIDADOS']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')
# Asegurarse de que las variables predictoras y objetivo estén en el formato correcto
# Imprimir los tipos de datos de las columnas del DataFrame completo
print(df_full.dtypes)

PERIODO                     int64
ANIO                     category
MES                      category
TRIMESTRE                category
ID_CAT1                  category
ID_CAT2                  category
ID_CAT3                  category
ID_BRAND                 category
SKU_SIZE                 category
CUSTOMER_ID              category
PRODUCT_ID               category
PLAN_PRECIOS_CUIDADOS    category
CUST_REQUEST_QTY            int64
CUST_REQUEST_TN           float64
TN                        float64
STOCK_FINAL               float64
MEDIA_MOVIL_3M            float64
DESVIO_MOVIL_3M           float64
TN_LAG_01                 float64
TN_LAG_02                 float64
TN_LAG_03                 float64
TN_LAG_06                 float64
TN_LAG_12                 float64
CLASE                     float64
CLASE_DELTA               float64
TN_DELTA_01               float64
TN_DELTA_02               float64
TN_DELTA_03               float64
TN_DELTA_06               float64
TN_DELTA_12   

In [16]:
# Usando este código como base para un modelo de regresión con LightGBM
# definir las variables predictoras y la variable objetivo indicando al modelo que las variables categóricas son de tipo 'category'

# Variables predictoras y objetivo
X = df_full[['PERIODO', 'CUSTOMER_ID','PRODUCT_ID', 'ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','PLAN_PRECIOS_CUIDADOS',
'CUST_REQUEST_QTY', 'TN','STOCK_FINAL','MEDIA_MOVIL_3M','DESVIO_MOVIL_3M','TN_LAG_01','TN_LAG_02','TN_LAG_03','TN_LAG_06','TN_LAG_12',
'TN_DELTA_01','TN_DELTA_02','TN_DELTA_03','TN_DELTA_06','TN_DELTA_12']]
y = df_full['CLASE_DELTA']


In [22]:

# Separar train y validation respetando la secuencia temporal
periodo_split = 201905  # Definir el periodo de corte para la separación
X_train = X[X['PERIODO'] <= periodo_split]
y_train = y[X['PERIODO'] <= periodo_split]
X_val = X[X['PERIODO'] > periodo_split]
y_val = y[X['PERIODO'] > periodo_split]
# Imprimir las formas de los conjuntos de entrenamiento y validación
print(f"Forma de X_train: {X_train.shape}, Forma de y_train: {y_train.shape}")
print(f"Forma de X_val: {X_val.shape}, Forma de y_val: {y_val.shape}")
# Imprimir las primeras filas de los conjuntos de entrenamiento y validación
print("Primeras filas de X_train:")
print(X_train.head())
print("Primeras filas de y_train:")
print(y_train.head())
print("Primeras filas de X_val:")
print(X_val.head())
print("Primeras filas de y_val:")
print(y_val.head())



Forma de X_train: (8236011, 27), Forma de y_train: (8236011,)
Forma de X_val: (2714462, 27), Forma de y_val: (2714462,)
Primeras filas de X_train:
   PERIODO CUSTOMER_ID PRODUCT_ID  ANIO MES TRIMESTRE ID_CAT1 ID_CAT2 ID_CAT3  \
0   201801       10001      20001  2018   1         1       2      11      57   
1   201802       10001      20001  2018   2         1       2      11      57   
2   201803       10001      20001  2018   3         1       2      11      57   
3   201804       10001      20001  2018   4         2       2      11      57   
4   201805       10001      20001  2018   5         2       2      11      57   

  ID_BRAND  ...  TN_LAG_01  TN_LAG_02  TN_LAG_03  TN_LAG_06  TN_LAG_12  \
0        1  ...  131.27150  111.54944  222.11389  101.20711   99.43861   
1        1  ...   49.61857  131.27150  111.54944   43.33930  198.84365   
2        1  ...   88.44065   49.61857  131.27150  289.35024   92.46537   
3        1  ...  214.72336   88.44065   49.61857  222.11389   13.29728

In [ ]:

import lightgbm as lgb

# Crear los datasets de LightGBM
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features)

# Definir parámetros para regresión
params = { 
    'objective': 'regression_l1',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 255,
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Entrenar el modelo con validación y early stopping
model = lgb.train(
    params,
    train_data,
    num_boost_round=10000,
    valid_sets=[val_data],
    valid_names=['validation'],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=100)]
)

print("Modelo de regresión entrenado con validación temporal (train hasta 201909, valid 201910).")

Training until validation scores don't improve for 100 rounds
[100]	valid's l1: 0.0594171
[200]	valid's l1: 0.0563615
[300]	valid's l1: 0.0538119
[400]	valid's l1: 0.0523435
[500]	valid's l1: 0.0514812
[600]	valid's l1: 0.0509819
[700]	valid's l1: 0.0506489
[800]	valid's l1: 0.0503179
[900]	valid's l1: 0.0501116
[1000]	valid's l1: 0.0499758
[1100]	valid's l1: 0.049836
[1200]	valid's l1: 0.0496657
[1300]	valid's l1: 0.0495251
[1400]	valid's l1: 0.0494262
[1500]	valid's l1: 0.0493815
[1600]	valid's l1: 0.0493327
[1700]	valid's l1: 0.0493011
[1800]	valid's l1: 0.0492863
[1900]	valid's l1: 0.0492647
[2000]	valid's l1: 0.0492366
[2100]	valid's l1: 0.0492189
[2200]	valid's l1: 0.049201
[2300]	valid's l1: 0.0491853
[2400]	valid's l1: 0.0491737
[2500]	valid's l1: 0.0491553
[2600]	valid's l1: 0.0491487
[2700]	valid's l1: 0.0491431
[2800]	valid's l1: 0.0491354
[2900]	valid's l1: 0.049131
[3000]	valid's l1: 0.0491256
[3100]	valid's l1: 0.0491222
[3200]	valid's l1: 0.049115
[3300]	valid's l1: 0.04

In [ ]:
# Guardo el modelo entrenado
model.save_model('modelo_regresion_lgbm_lags_delta_promedio_desvio.txt')

In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select * from L_VM_COMPLETA where periodo = 201912" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

In [ ]:
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_pred_full[col] = df_pred_full[col].astype('category')


# Con el modelo entrenado, hacemos predicciones 
X_pred = df_pred_full[['PERIODO', 'CUSTOMER_ID','PRODUCT_ID', 'ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','PLAN_PRECIOS_CUIDADOS',
'CUST_REQUEST_QTY', 'TN','STOCK_FINAL','MEDIA_MOVIL_3M','DESVIO_MOVIL_3M','TN_LAG_01','TN_LAG_02','TN_LAG_03','TN_LAG_06','TN_LAG_12']]



In [ ]:

predictions = model.predict(X_pred)
# Agregar las predicciones al DataFrame original
df_pred_full['PREDICCIONES'] = predictions
# Imprimir las primeras filas del DataFrame con las predicciones
print(df_pred_full.head())
# Guardar el DataFrame con las predicciones en un archivo CSV
df_pred_full.to_csv('predicciones.csv', index=False)
# Imprimir el número de filas y columnas del DataFrame con las predicciones
print(f"Número de filas: {df_pred_full.shape[0]}, Número de columnas: {df_pred_full.shape[1]} con predicciones.")



In [ ]:
# Con el DataFrame de predicción, actualizamos la base de datos
# el criterio es actualizar la tabla L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones
# la columnna PREDICCIONES se debe actualizar con los nuevos valores
# la clave primaria es (PERIODO, CUSTOMER_ID, PRODUCT_ID)
# Hacer commit cada 10000 filas para evitar problemas de memoria

# Conectar a la base de datos para actualizar los datos de predicción
conn = cx_Oracle.connect(user="pc", password="p201404", dsn="siatchdesa")
# Crear un cursor para ejecutar las actualizaciones
cursor = conn.cursor()
update_query = """
    UPDATE L_VM_COMPLETA_PREDICCIONES
    SET PREDICCION = NULL
"""
cursor.execute(update_query)
# Hacer commit para aplicar el cambio de NULL
conn.commit()


In [ ]:


# Imprimir mensaje de inicio de actualización
print("Iniciando actualización de la tabla L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones.")

# Iterar sobre las filas del DataFrame con las predicciones
for index, row in df_pred_full.iterrows():
    periodo = row['PERIODO']
    customer_id = row['CUSTOMER_ID']
    product_id = row['PRODUCT_ID']
    prediccion = row['PREDICCIONES']
    
    # Actualizar la tabla L_DATOS_PREDICCION con la nueva predicción
    update_query = """
        UPDATE L_VM_COMPLETA_PREDICCIONES
        SET PREDICCION = :prediccion
        WHERE PERIODO = :periodo AND CUSTOMER_ID = :customer_id AND PRODUCT_ID = :product_id
    """
    cursor.execute(update_query, {'prediccion': prediccion, 'periodo': periodo, 'customer_id': customer_id, 'product_id': product_id})  
    # Hacer commit cada 10000 filas para evitar problemas de memoria
    if index % 10000 == 0:
        conn.commit()
        print(f"Actualizadas {index} filas de L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones.")
# Confirmar los cambios en la base de datos
conn.commit()
# Cerrar el cursor y la conexión
cursor.close()
conn.close()
# Imprimir mensaje de finalización
print("Actualización de la tabla  completada con las nuevas predicciones.")
